In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import nltk
stopwords = nltk.corpus.stopwords.words('english')


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles.pql"
article_df = pd.read_pickle(data_path)

In [5]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/chemistry.pql"
data = None
che_categories = pd.read_pickle(data_path)

In [6]:
data = None
data = che_categories.merge(article_df, 
                            how = 'inner',
                            on = 'ArticleID')

In [8]:
data['title_without_punct'] = data['Title'].progress_apply(lambda x: 
                                                           str.lower(''.join(ch for ch in str(x) if category(ch)[0] != 'P')))
data["title_without_stopwords"] = data['title_without_punct'].progress_apply(lambda x: [word for word in x.split() if word not in stopwords])

Progress: 100%|██████████| 5285784/5285784 [02:19<00:00, 37794.87it/s]


In [9]:
len(data)

5285784

In [10]:
data.Label.unique()

array(['Chemistry, Applied', 'Chemistry, Organic',
       'Chemistry, Multidisciplinary', 'Chemistry, Physical',
       'Chemistry, Inorganic & Nuclear', 'Chemistry, Medicinal',
       'Chemistry, Analytical'], dtype=object)

In [11]:
data.groupby('Label').size()

Label
Chemistry, Analytical              586860
Chemistry, Applied                 349407
Chemistry, Inorganic & Nuclear     429712
Chemistry, Medicinal               296573
Chemistry, Multidisciplinary      1625520
Chemistry, Organic                 716608
Chemistry, Physical               1281104
dtype: int64

In [12]:
data.head()

,ArticleID,Type,Label,PubYear,Doctypes,Title,Journal,Journal Abbreviation ISO,title_without_punct,title_without_stopwords
0,A1981LJ47400021,ASCA traditional subject,"Chemistry, Applied",1981,Article,PROCESSING AND UTILIZATION OF BY-PRODUCTS FROM...,JOURNAL OF THE AMERICAN OIL CHEMISTS SOCIETY,J. Am. Oil Chem. Soc.,processing and utilization of byproducts from ...,"[processing, utilization, byproducts, soy, oil..."
1,A1981LC78900005,ASCA traditional subject,"Chemistry, Applied",1981,Article,COMPARATIVE METABOLISM OF ZEARALENONE AND TRAN...,FOOD AND COSMETICS TOXICOLOGY,,comparative metabolism of zearalenone and tran...,"[comparative, metabolism, zearalenone, transmi..."
2,A1981NZ41100025,ASCA traditional subject,"Chemistry, Applied",1981,Note,DEVELOPMENT OF A MATHEMATICAL-MODEL AND DETERM...,JOURNAL OF APPLIED CHEMISTRY OF THE USSR,,development of a mathematicalmodel and determi...,"[development, mathematicalmodel, determination..."
3,A1981KY62300009,ASCA traditional subject,"Chemistry, Applied",1981,Article,CATALYZING THE ENERGY-BALANCE OF CAT CRACKING,CHEMTECH,Chemtech,catalyzing the energybalance of cat cracking,"[catalyzing, energybalance, cat, cracking]"
4,A1981MT10600006,ASCA traditional subject,"Chemistry, Applied",1981,Note,THE NON-SELECTIVE ESTERIFICATION OF SECONDARY ...,CHEMISTRY & INDUSTRY,Chem. Ind.,the nonselective esterification of secondary a...,"[nonselective, esterification, secondary, alco..."


In [ ]:
data_grouped_by_year = data.groupby('PubYear').agg({'title_without_stopwords': 'sum'}).reset_index()

In [ ]:
data_grouped_by_year.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/title-words-grouped-by-year.pql")